# Bayes starting point search

The idea here is to use an approximation of Bayesian model comparison to identify the starting point `x1` where the linar part of a given time series transitions toa sigmoidal part

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import random

## Generate synthetic data

To be able to assess the quality of our model we first generate a large number of synthetic traces plus the correct `x1`.

In [ ]:
random.seed(123)  # For repeatability

N = 6  # number of traces
data = pd.DataFrame()

for i in range(N):
    S = random.randint(170, 400)  # number of samples
    y0 = random.uniform(1, 20)  # offset
    s = random.uniform(-0.2, 0.2)  # slope of linear part
    std = random.uniform(3, 15)  # noise
    x12 = random.randint(80, S - 80)  # halfway-point of sigmoid
    xSigWidth = random.uniform(0.5, 30)  # width scaling factor of sigmoid
    ySig = random.uniform(20, 300)  # height of sigmoid
    xTrue = x12 - xSigWidth  # The "true" starting point

    x = np.arange(start=0, stop=S)
    y = (
        y0
        + s * x
        + ySig * (0.5 + 0.5 * np.tanh((x - x12) / xSigWidth))
        + np.random.normal(scale=std, size=S)
    )
    data = data.append(
        {"x": x, "y": y, "S": S, "x12": x12, "xSigWidth": xSigWidth, "xTrue": xTrue},
        ignore_index=True,
    )

In [ ]:
data.head()

In [ ]:
fig = plt.figure(figsize=(9.5, 4))

for i in range(min(N, 4)):
    trace = data.loc[i]
    p = fig.add_subplot(1, 4, i + 1)
    p.set(
        xlabel="sample", title=f"trace #{i+1}",
    )
    p.plot(trace.x, trace.y)
    p.axvline(x=trace.xTrue, linewidth=1, color="r")

## Compare fits for a single trace

In [ ]:
from scipy.stats import linregress
from scipy.optimize import curve_fit


def cut_raw_log_bf(x, y, limit, verbose=False):
    return raw_log_bayes_factor(x[:limit], y[:limit], verbose=verbose)


def lin_plus_sigmoid(x, y0, s, ySig, x12, xSigWidth):
    return y0 + s * x + ySig * (0.5 + 0.5 * np.tanh((x - x12) / xSigWidth))


def raw_log_bayes_factor(x, y, verbose=False):
    """This is the logBF without the constant factors."""
    variance_noise = np.var(y[:50])
    if verbose:
        print(f"variance_noise = {variance_noise} (std of {np.sqrt(variance_noise)})")

    slope, intercept, r_value, p_value, std_err0 = linregress(x, y)
    variance0 = np.var([intercept + x * slope for x in x] - y)
    if verbose:
        print(f"variance0 = {variance0} (std of {np.sqrt(variance0)})")

    try:
        popt, pcov = curve_fit(
            lin_plus_sigmoid,
            x,
            y,
            p0=[intercept, slope, 2 * (max(y) - min(y)), len(x) / 2, 5],
            bounds=([-500, -100, 1, 5, 0.01], [500, 100, 5000, 5000, 100]),
        )
        variance1 = np.var([lin_plus_sigmoid(x, *popt) for x in x] - y)
        if variance1 == float("inf") or variance1 == float("-inf"):
            if verbose:
                print("Warning: variance1 is infinite")
            raise
        if verbose:
            print(f"variance1 = {variance1} (std of {np.sqrt(variance1)})")

        bf = len(x) * (variance0 - variance1) / variance_noise
        if verbose:
            print(f"bf = {bf}")
        return (bf, (intercept, slope), tuple(popt))
    except:
        print("Warning: Failed to find sigmoidal fit.")
        return (None, (intercept, slope), (None, None, None, None, None))

In [ ]:
trace = data.loc[0]
# raw_log_bayes_factor(trace.x, trace.y)
cut_raw_log_bf(trace.x,trace.y,100)

In [ ]:
trace = data.loc[1]

fig = plt.figure(figsize=(9.5, 2 * 4))

# Plot trace itself again
p = fig.add_subplot(2, 2, 1)
p.set(title="amplitude")
p.set_xlim([0, len(trace.x)])
p.plot(trace.x, trace.y)
p.axvline(x=trace.xTrue, linewidth=1, color="r")

# Plot Bayes factor
xs = np.arange(start=50, stop=len(trace.x))
bf_trace = [cut_raw_log_bf(trace.x, trace.y, i)[0] for i in xs]
p = fig.add_subplot(2, 2, 2)
p.set(title="Bayes factor",)
p.set_xlim([0, len(trace.x)])
p.set_ylim([-100, 1000])
p.plot(xs, bf_trace)
p.axvline(x=trace.xTrue, linewidth=1, color="r")

# Find fit at specific point
x_limit = 150
(bf, (intercept, slope), sig_params) = cut_raw_log_bf(
    trace.x, trace.y, x_limit, verbose=True
)
print(f"At x_limit = {x_limit}, BF = {bf}")

if bf is not None:
    # Plot linear fit
    ys_fit = intercept + slope * trace.x
    p = fig.add_subplot(2, 2, 3)
    p.set(title="Linear fit")
    p.set_xlim([0, len(trace.x)])
    #     p.plot(trace.x, trace.y)
    p.plot(trace.x[:x_limit], trace.y[:x_limit])
    p.plot(trace.x[x_limit:], trace.y[x_limit:], color="0.8")
    p.plot(trace.x, ys_fit)
    p.axvline(x=trace.xTrue, linewidth=1, color="r")
    p.axvline(x=x_limit, linewidth=1, color="0.7")

    # Plot sigmoidal fit
    ys_fit = [lin_plus_sigmoid(x, *sig_params) for x in trace.x]
    p = fig.add_subplot(2, 2, 4)
    p.set(title="Sigmoidal fit")
    p.set_xlim([0, len(trace.x)])
    #     p.plot(trace.x, trace.y)
    p.plot(trace.x[:x_limit], trace.y[:x_limit])
    p.plot(trace.x[x_limit:], trace.y[x_limit:], color="0.8")
    p.plot(trace.x, ys_fit)
    p.axvline(x=trace.xTrue, linewidth=1, color="r")
    p.axvline(x=x_limit, linewidth=1, color="0.7")

## Compute Bayes factor traces

In [ ]:
fig = plt.figure(figsize=(9.5, 4))

for i in range(min(N, 4)):
    trace = data.loc[i]
    xs = np.arange(start=50, stop=len(trace.x))
    bf_trace = [cut_raw_log_bf(trace.x, trace.y, i)[0] for i in xs]

    p = fig.add_subplot(1, 4, i + 1)
    p.set(
        xlabel="sample", title=f"trace #{i+1}",
    )
    p.plot(xs, bf_trace)
    p.set_ylim([-5, 100])
    p.axvline(x=trace.xTrue, linewidth=1, color="r")
    p.axhline(y=3, linewidth=1, color="g")

In [ ]:
def add_bf_trace(row):
    bf_x = np.arange(start=50, stop=len(row.x))
    bf_y = np.array([cut_raw_log_bf(row.x, row.y, i)[0] for i in bf_x])

    # Remove points where no fit was possible
    bf_trace = np.array(list(zip(bf_x, bf_y)))
    row["bf_trace"] = bf_trace[bf_trace[:, 1] != None]

    print("One trace done.")
    return row


data_with_bfs = data.apply(add_bf_trace, axis=1)
data_with_bfs.head()

In [ ]:
data_with_bfs.head()

## Evaluate mean error depending on "Bayes offset"

In [ ]:
# Scan through the following values of the Bayes offset
cs = np.linspace(start=0, stop=40, num=20)

def find_last_root(trace, thresh=0):
    # We reverse the trace since we want to start from the end
    t2 = np.flip(trace, 0)
    # TODO Actually interpolate
    #     last_x, _last_y = None, None
    for x, y in t2:
        if y <= thresh:
            return x
    return 0

def error(bf_trace, xTrue, const):
    return abs(xTrue - find_last_root(bf_trace, const))

maes = []

for c in cs:
    errors = [error(row.bf_trace, row.xTrue, c) for i, row in data_with_bfs.iterrows()]
    mae = sum(errors) / len(errors)
    maes.append(mae)

In [ ]:
maes

In [ ]:
fig = plt.figure()
p = fig.add_subplot(1, 1, 1)
p.set(
    xlabel="Bayes offset", ylabel=f"mean absolute error",
)
p.plot(cs, maes)